# Incremental Collaborative Filtering (ICR) - Algorithm experiments

Movielens 100k data set: https://maxhalford.github.io/files/datasets/ml_100k.zip

In [41]:
import csv
import numpy as np

In [177]:
data = []
with open('ml_100k.csv', 'r') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf, delimiter='\t') 
        for row in csvReader: 
            #add this python dict to array
            data.append(row)

# Get a smaller data set to experiment on
test = data[0:500].copy()

In [182]:
user_ratings = {} # Dict to store ratings
user_meta = {} # Dict to cache users info on number of ratings and average ratings
user_pair_meta = {} # Dict to cache info on factors calculated for pairs of users (B, C, D, sum of ratings to co-rated items)
# first_run = True

# Simulate streaming data by reading user ratings one by one
for stream in test:
    user, item, rating = stream['user'], stream['item'], float(stream['rating'])
    
    # initialize new user
    if user not in user_meta.keys():
        user_ratings[user] = {} # initializes user in ratings dict
        user_meta[user] = {'q': 0, 'avg.rating': 0} # initializes user in meta dict, assign number of items user has rated and avg rating of user
        
        # initializes pairs of existing users with new user in user_pair_meta dict
        for u in user_meta.keys(): 
            if u == user:
                continue
            user_pair_meta[(u, user)] = {'B': 0, 'C': 0, 'D': 0}
            user_pair_meta[(u, user)]['sum.co_ratings'] = {u: 0, user: 0}
            
    # if user submits a new rating
    if item not in user_ratings[user].keys():        
        q = user_meta[user]['q'] # gets number of items user has rated
        new_avg = round( ( rating/( q+1 ) ) + ( q/( q+1 ) )*user_meta[user]['avg.rating'], 2 ) # calculates new avg rating for active user
        delta_avg = new_avg - user_meta[user]['avg.rating'] # difference of user's previous and current avg rating        

    # if user updates a rating
    else:
        print('update')
        delta_rating = rating - user_ratings[user][item] # difference of user's previous and current rating for item
        q = user_meta[user]['q'] # gets number of items user has rated
        new_avg = round( delta_rating/q + user_meta[user]['avg.rating'], 2 ) # calculates new avg rating for active user
        delta_avg = new_avg - user_meta[user]['avg.rating'] # difference of user's previous and current avg rating        

        
#     if first_run:
    for u in user_meta.keys(): 
        if u == user:
            continue
        for i in user_ratings[user].keys():
            if i in user_ratings[u].keys():
                print(user, u, i, user_meta[user]['q'] )
                
    user_meta[user]['avg.rating'] = new_avg # updates avg rating
    user_ratings[user][item] = rating # updates rating given by user to item
    user_meta[user]['q'] += 1 # updates number of items user has rated
    
#     first_run = False

712 259 173 16
712 259 173 17
712 259 173 18
712 259 173 19
712 259 173 20
712 259 173 21
712 259 173 22
712 259 173 23
712 259 173 24
712 259 173 25
712 259 173 26
712 259 173 27
712 259 173 28
712 259 173 29
712 259 173 30
712 259 173 31
712 259 173 32
712 259 173 33
712 259 173 34
712 259 173 35
712 259 173 36
712 259 173 37
712 259 173 38
712 259 173 39
712 259 173 40
712 259 173 41
712 259 173 42
712 259 173 43
712 259 173 44
712 259 173 45
712 259 173 46
712 259 173 47
712 259 173 48
712 259 173 49
712 259 173 50
712 259 173 51
712 259 173 52
712 259 173 53
712 259 173 54
712 259 173 55
712 259 173 56
712 259 173 57
712 259 173 58
712 259 173 59
712 259 210 59
712 259 173 60
712 259 210 60
712 259 173 61
712 259 210 61
712 259 173 62
712 259 210 62
712 259 173 63
712 259 210 63
712 259 173 64
712 259 210 64
712 259 173 65
712 259 210 65
712 259 173 66
712 259 210 66
712 259 173 67
712 259 210 67
712 259 173 68
712 259 210 68
712 259 173 69
712 259 210 69
712 259 173 70
712 259 21